# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-22291c7aca-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# GEO Workflow Series ("American Architects") 

Consider the following exploratory information need:

> You want to study the most prolific american architects

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P106`    | occupation    | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wd:Q5604`    | Frank Lloyd Wright     | node      |
| `wd:Q30`      | U.S.A.        | node |
| `wd:Q42973`   | architect     | node |
| `wd:Q2081276` | Pettit Memorial Chapel        | node |
| `wd:Q2977`    | Cathedral     | node |



Also consider

```
?p wdt:P27 wd:Q30  . 
?p wdt:P106 wd:Q42973  . 
```

is the BGP to retrieve all **american architects**

## Workload Goals

1. Identify the BGP to obtain the notable works of an architect and their location

2. Identify the BGP that connect a building with its architect, type, and architectural style

3. Which architect is the most prolific in the U.S. and outside?

4. Analyze the location of the buildings designed by the american architects
 
   4.1 Which styles exists in the U.S.? Which type of building?
   
   4.2 In which U.S. states there are more notable buildings?
   
   4.3 Which U.S. state contains the largest number of building designed by U.S. architects


In [3]:
# start your workflow here

In [2]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P27 wd:Q30  . 
?p wdt:P106 wd:Q42973  . 

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '3354')]


1

## Task 1 - Identify the BGP to obtain the notable works of an architect and their location

#### I search all properties of American Architects

In [ ]:
# Architects - p - obj

queryString = """
SELECT DISTINCT ?prop ?pname WHERE { 

?p wdt:P27 wd:Q30. 
?p wdt:P106 wd:Q42973. 
?p ?prop ?obj.

?prop <http://schema.org/name> ?pname .
} 
"""

print("Results")
run_query(queryString)

In [8]:
# Sub - p - Architects

queryString = """
SELECT DISTINCT ?prop ?pname WHERE { 

?p wdt:P27 wd:Q30. 
?p wdt:P106 wd:Q42973. 
?sub ?prop ?p.

?prop <http://schema.org/name> ?pname.
} 

"""

print("Results")
run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1423'), ('pname', 'template has topic')]
[('prop', 'http://www.wikidata.org/prop/direct/P1268'), ('pname', 'represents')]
[('prop', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('prop', 'http://www.wikidata.org/prop/direct/P1038'), ('pname', 'relative')]
[('prop', 'http://www.wikidata.org/prop/direct/P1040'), ('pname', 'film editor')]
[('prop', 'http://www.wikidata.org/prop/direct/P1066'), ('pname', 'student of')]
[('prop', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('prop', 'http://www.wikidata.org/prop/direct/P110'), ('pname', 'illustrator')]
[('prop', 'http://www.wikidata.org/prop/direct/P112'), ('pname', 'founded by')]
[('prop', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of')]
[('prop', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by')]
[('prop', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('prop', 'http://www.wi

96

#### Let's try with "Designed by"

In [ ]:
# Designed by: P287

queryString = """
SELECT DISTINCT ?subname WHERE { 

?p wdt:P27 wd:Q30. 
?p wdt:P106 wd:Q42973. 
?sub wdt:P287 ?p.

?sub <http://schema.org/name> ?subname.
} 

"""
print("Results")
run_query(queryString)

#### Not happy, let's try with "notable work" property of the architects

In [ ]:
# Notable Work: P800

queryString = """
SELECT DISTINCT ?workname WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.

?work <http://schema.org/name> ?workname.
} 

"""

print("Results")
run_query(queryString)

#### Very better now i inspect all properties of buildings

In [24]:
# Building - p - obj

queryString = """
SELECT DISTINCT ?prop ?propname (COUNT(?obj) AS ?number) WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?work ?prop ?obj.

?prop <http://schema.org/name> ?propname
} GROUP BY ?prop ?propname
ORDER BY DESC (?number)

"""
print("Results")
run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('propname', 'instance of'), ('number', '1177')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('propname', 'country'), ('number', '787')]
[('prop', 'http://www.wikidata.org/prop/direct/P131'), ('propname', 'located in the administrative territorial entity'), ('number', '779')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('propname', 'image'), ('number', '773')]
[('prop', 'http://www.wikidata.org/prop/direct/P84'), ('propname', 'architect'), ('number', '677')]
[('prop', 'http://www.wikidata.org/prop/direct/P646'), ('propname', 'Freebase ID'), ('number', '668')]
[('prop', 'http://www.wikidata.org/prop/direct/P373'), ('propname', 'Commons category'), ('number', '594')]
[('prop', 'http://www.wikidata.org/prop/direct/P1435'), ('propname', 'heritage designation'), ('number', '526')]
[('prop', 'http://www.wikidata.org/prop/direct/P571'), ('propname', 'inception'), ('number', '463')]
[('prop', 'http://www.wikidata

770

In [ ]:
#Sub - p - Building

queryString = """
SELECT DISTINCT ?prop ?propname WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?sub ?prop ?work.

?prop <http://schema.org/name> ?propname
} 

"""
print("Results")
run_query(queryString)

#### I return for each building the country where it is located

In [ ]:
# Country: P17

queryString = """
SELECT DISTINCT ?workname ?country WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?work wdt:P17 ?ctr.

?work <http://schema.org/name> ?workname.
?ctr <http://schema.org/name> ?country.
} 

"""
print("Results")
run_query(queryString)

## Task 2 - Identify the BGP that connect a building with its architect, type, and architectural style

#### I found property "architect" of a building that connects to its architect

In [ ]:
# Architect: P84

queryString = """
SELECT DISTINCT ?workname ?architect WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?work wdt:P84 ?a.

?work <http://schema.org/name> ?workname.
?a <http://schema.org/name> ?architect.
} 

"""
print("Results")
run_query(queryString)

#### To get the type of the building i use "instance of"

In [4]:
# Instance of: P31

queryString = """
SELECT DISTINCT ?workname ?type WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?work wdt:P31 ?t.

?work <http://schema.org/name> ?workname.
?t <http://schema.org/name> ?type.
} 

"""
print("Results")
run_query(queryString)

Results
[('workname', "Washington Governor's Mansion"), ('type', 'building')]
[('workname', 'Earls Court Exhibition Centre'), ('type', 'sports venue')]
[('workname', 'Earls Court Exhibition Centre'), ('type', 'convention center')]
[('workname', 'Earls Court Exhibition Centre'), ('type', 'music venue')]
[('workname', 'Castle Hill'), ('type', 'estate')]
[('workname', 'Detroit Olympia'), ('type', 'arena')]
[('workname', 'Country Club Plaza'), ('type', 'neighborhood')]
[('workname', 'Arkansas State Capitol'), ('type', 'capitol building')]
[('workname', 'Arkansas State Capitol'), ('type', 'tourist attraction')]
[('workname', 'U.S. Bank Building'), ('type', 'building')]
[('workname', 'Kennedy-Warren Apartment Building'), ('type', 'low-rise building')]
[('workname', 'Kennedy-Warren Apartment Building'), ('type', 'apartment building')]
[('workname', 'Rec Hall'), ('type', 'arena')]
[('workname', 'Audubon Terrace'), ('type', 'building complex')]
[('workname', 'Church of Saint Stephen'), ('type',

1127

#### Architectural Style

In [ ]:
# Architectural Style: P149

queryString = """
SELECT DISTINCT ?workname ?style WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?work wdt:P149 ?s.

?work <http://schema.org/name> ?workname.
?s <http://schema.org/name> ?style.
} 

"""
print("Results")
run_query(queryString)

## Task 3 - Which architect is the most prolific in the U.S. and outside?

In [ ]:
#### I count only buildings in the US

In [32]:
# Country: P17

queryString = """
SELECT DISTINCT ?architect (count(?work) as ?number) WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?work wdt:P17 wd:Q30.

?p <http://schema.org/name> ?architect.
} 
GROUP BY (?architect)
ORDER BY DESC (?number)
LIMIT 1
"""
print("Results")
run_query(queryString)

Results
[('architect', 'Bruce Price'), ('number', '13')]


1

#### ... and pick every country except US

In [37]:
# Country: P17

queryString = """
SELECT DISTINCT ?architect (count(?work) as ?number) WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?work wdt:P17 ?ctr.

?p <http://schema.org/name> ?architect.

FILTER (?ctr != wd:Q30)
} 
GROUP BY (?architect)
ORDER BY DESC (?number)
LIMIT 1
"""
print("Results")
run_query(queryString)

Results
[('architect', 'Cornelia Oberlander'), ('number', '11')]


1

## Task 4 - Analyze the location of the buildings designed by the american architects

#### First of all let's count how many building per country are designed by american architects

In [3]:
# Country: P17
queryString = """
SELECT DISTINCT ?country (COUNT(DISTINCT ?work) AS ?number) WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?work wdt:P17 ?ctr.

?ctr <http://schema.org/name> ?country.
} GROUP BY (?country)
ORDER BY DESC (?number)

"""
print("Results")
run_query(queryString)

Results
[('country', 'United States of America'), ('number', '653')]
[('country', 'Canada'), ('number', '30')]
[('country', 'Germany'), ('number', '11')]
[('country', 'United Kingdom'), ('number', '9')]
[('country', 'Spain'), ('number', '9')]
[('country', 'France'), ('number', '9')]
[('country', "People's Republic of China"), ('number', '9')]
[('country', 'Saudi Arabia'), ('number', '3')]
[('country', 'Netherlands'), ('number', '3')]
[('country', 'United Arab Emirates'), ('number', '3')]
[('country', 'Serbia'), ('number', '2')]
[('country', 'Japan'), ('number', '2')]
[('country', 'Austria'), ('number', '2')]
[('country', 'Finland'), ('number', '2')]
[('country', 'Taiwan'), ('number', '2')]
[('country', 'Italy'), ('number', '2')]
[('country', 'Panama'), ('number', '1')]
[('country', 'Malaysia'), ('number', '1')]
[('country', 'Norway'), ('number', '1')]
[('country', 'Peru'), ('number', '1')]
[('country', 'Poland'), ('number', '1')]
[('country', 'Croatia'), ('number', '1')]
[('country', '

31

### Task 4.1 Which styles exists in the U.S.? Which type of building?

#### Let's count the style of the buildings in the US 

In [7]:
# Architectural Style: P149

queryString = """
SELECT DISTINCT ?style (COUNT(?work) AS ?number) WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.

?work wdt:P17 wd:Q30;   
      wdt:P149 ?s.

?s <http://schema.org/name> ?style.
} GROUP BY (?style)
ORDER BY DESC (?number)

"""
print("Results")
run_query(queryString)

Results
[('style', 'Gothic Revival'), ('number', '25')]
[('style', 'Beaux-Arts'), ('number', '24')]
[('style', 'modern architecture'), ('number', '23')]
[('style', 'Renaissance Revival architecture'), ('number', '22')]
[('style', 'Neoclassical architecture'), ('number', '20')]
[('style', 'Art Deco'), ('number', '20')]
[('style', 'Romanesque Revival architecture'), ('number', '15')]
[('style', 'Greek Revival architecture'), ('number', '14')]
[('style', 'Mediterranean Revival architecture'), ('number', '12')]
[('style', 'International Style'), ('number', '11')]
[('style', 'National Park Service rustic'), ('number', '9')]
[('style', 'postmodern architecture'), ('number', '9')]
[('style', 'Spanish Colonial Revival architecture'), ('number', '8')]
[('style', 'neoclassicism'), ('number', '5')]
[('style', 'Mission Revival architecture'), ('number', '5')]
[('style', 'Colonial Revival architecture'), ('number', '5')]
[('style', 'Streamline Moderne'), ('number', '5')]
[('style', 'Art Deco archit

73

#### I use the property "heritage designation" (wdt:P1435) to find how many sites have cultural relevance

In [10]:
# Heritage Designation P1435

queryString = """
SELECT DISTINCT ?type ?typename WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.

?work wdt:P17 wd:Q30;   
      wdt:P1435 ?type.

?type <http://schema.org/name> ?typename.
} GROUP BY (?style)
ORDER BY DESC (?number)

"""
print("Results")
run_query(queryString)

Results
[('type', 'http://www.wikidata.org/entity/Q1129142'), ('typename', 'NRHP contributing property')]
[('type', 'http://www.wikidata.org/entity/Q70483386'), ('typename', 'Mississippi Landmark')]
[('type', 'http://www.wikidata.org/entity/Q56572098'), ('typename', 'Seattle landmark')]
[('type', 'http://www.wikidata.org/entity/Q43113623'), ('typename', 'part of UNESCO World Heritage Site')]
[('type', 'http://www.wikidata.org/entity/Q4742563'), ('typename', "America's Most Endangered Historic Places")]
[('type', 'http://www.wikidata.org/entity/Q16152996'), ('typename', 'California Point of Historical Interest')]
[('type', 'http://www.wikidata.org/entity/Q5335134'), ('typename', 'Los Angeles Historic-Cultural Monument')]
[('type', 'http://www.wikidata.org/entity/Q1459900'), ('typename', 'Tentative World Heritage Site')]
[('type', 'http://www.wikidata.org/entity/Q106945314'), ('typename', 'National Treasure')]
[('type', 'http://www.wikidata.org/entity/Q73198230'), ('typename', 'District 

23

#### Now i count the buldings for each type. I use the properties "heritage designation" and "use" (wdt:P366)

In [22]:
# Heritage Designation P1435
# Use P366

queryString = """
SELECT DISTINCT  ?typename ?number WHERE { 
{
#Subquery: count how many US building have cultural relevance
SELECT DISTINCT ?typename (COUNT (*) AS ?number) WHERE{
?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.

?work wdt:P17 wd:Q30;   
      ?type ?o.
FILTER(?type=wdt:P1435)
?type <http://schema.org/name> ?typename.
}
} 
UNION
{
#Subquery: count the buildings by their reported use
SELECT DISTINCT ?typename (COUNT (?work) AS ?number) WHERE{
?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.

?work wdt:P17 wd:Q30;   
      wdt:P366 ?type.

?type <http://schema.org/name> ?typename.
}
GROUP BY (?typename)
}
}ORDER BY DESC (?number)
"""
print("Results")
run_query(queryString)

Results
[('typename', 'heritage designation'), ('number', '479')]
[('typename', 'office'), ('number', '16')]
[('typename', 'hotel'), ('number', '5')]
[('typename', 'official residence'), ('number', '4')]
[('typename', 'office building'), ('number', '2')]
[('typename', 'public aquarium'), ('number', '2')]
[('typename', 'visitor centre'), ('number', '2')]
[('typename', 'residence'), ('number', '2')]
[('typename', 'government office'), ('number', '2')]
[('typename', 'community center'), ('number', '2')]
[('typename', 'historic house museum'), ('number', '2')]
[('typename', 'lecture'), ('number', '1')]
[('typename', 'junior college'), ('number', '1')]
[('typename', 'department store'), ('number', '1')]
[('typename', 'mixed-use building'), ('number', '1')]
[('typename', 'library branch'), ('number', '1')]
[('typename', 'restaurant'), ('number', '1')]
[('typename', 'community house'), ('number', '1')]
[('typename', 'female seminary'), ('number', '1')]
[('typename', 'military housing'), ('num

40

#### Ok it was a strange query by counting a predicate and different entities but the result is good.

### Task 4.2 In which U.S. states there are more notable buildings?

#### Let's try to get all locations of the buildings with "located in administrative district"(wdt:P131)

In [26]:
# located in administrative district: P131

queryString = """
SELECT DISTINCT  ?work ?loc ?place WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?work wdt:P17 wd:Q30;
      wdt:P131 ?loc.

?loc <http://schema.org/name> ?place.
} 
"""
print("Results")
run_query(queryString)

Results
[('work', 'http://www.wikidata.org/entity/Q756433'), ('loc', 'http://www.wikidata.org/entity/Q51682'), ('place', 'Glendale')]
[('work', 'http://www.wikidata.org/entity/Q11208'), ('loc', 'http://www.wikidata.org/entity/Q107126'), ('place', 'Arlington County')]
[('work', 'http://www.wikidata.org/entity/Q3584637'), ('loc', 'http://www.wikidata.org/entity/Q11299'), ('place', 'Manhattan')]
[('work', 'http://www.wikidata.org/entity/Q6499849'), ('loc', 'http://www.wikidata.org/entity/Q1345'), ('place', 'Philadelphia')]
[('work', 'http://www.wikidata.org/entity/Q634221'), ('loc', 'http://www.wikidata.org/entity/Q16555'), ('place', 'Houston')]
[('work', 'http://www.wikidata.org/entity/Q5177248'), ('loc', 'http://www.wikidata.org/entity/Q41819'), ('place', 'Kansas City')]
[('work', 'http://www.wikidata.org/entity/Q5591456'), ('loc', 'http://www.wikidata.org/entity/Q43668'), ('place', 'Louisville')]
[('work', 'http://www.wikidata.org/entity/Q7492097'), ('loc', 'http://www.wikidata.org/ent

643

#### I retrieve the properties of the locations

In [25]:
queryString = """
SELECT DISTINCT ?pname (COUNT(?obj) AS ?number) WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?work wdt:P17 wd:Q30;
      wdt:P131 ?loc.

?loc ?b ?obj.
      
?b <http://schema.org/name> ?pname.

} 
GROUP BY (?pname)
ORDER BY DESC (?number)
"""
print("Results")
run_query(queryString)

Results
[('pname', 'contains administrative territorial entity'), ('number', '9826')]
[('pname', 'twinned administrative body'), ('number', '4980')]
[('pname', 'shares border with'), ('number', '3826')]
[('pname', 'postal code'), ('number', '2272')]
[('pname', 'language used'), ('number', '1940')]
[('pname', 'described by source'), ('number', '1861')]
[('pname', 'instance of'), ('number', '1544')]
[('pname', 'located in time zone'), ('number', '1374')]
[('pname', 'located in or next to body of water'), ('number', '1186')]
[('pname', 'GNIS ID'), ('number', '1046')]
[('pname', 'area'), ('number', '973')]
[('pname', 'owner of'), ('number', '937')]
[('pname', 'local dialing code'), ('number', '850')]
[('pname', 'different from'), ('number', '779')]
[('pname', 'part of'), ('number', '727')]
[('pname', 'located in the administrative territorial entity'), ('number', '688')]
[('pname', 'Curlie ID'), ('number', '685')]
[('pname', 'population'), ('number', '684')]
[('pname', 'GeoNames ID'), ('nu

432

#### I inspect "Oregon" to get how is defined a US state

In [26]:
queryString = """
SELECT DISTINCT ?obj ?objname WHERE { 

wd:Q824 wdt:P31 ?obj.

?obj <http://schema.org/name> ?objname.
} 

"""
print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q35657'), ('objname', 'U.S. state')]


1

#### I count how many locations that i got before are US states

In [30]:
# US State Q35657

queryString = """
SELECT DISTINCT  (COUNT(?work) as ?number)  WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?work wdt:P17 wd:Q30;
      wdt:P131 ?loc.

?loc wdt:P31 wd:Q35657.

?loc <http://schema.org/name> ?place.

} 

"""
print("Results")
run_query(queryString)

Results
[('number', '153')]


1

#### ...Ony 153 out of 653 buildings, few data

#### I noticed that the property "located in administrative district" is recursive so i can try to go more in depth until i find a US state if reported for each building

In [30]:
# US State Q35657

queryString = """
SELECT DISTINCT  (COUNT(?work) as ?number)  WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?work wdt:P17 wd:Q30;
      wdt:P131+ ?loc.

?loc wdt:P31 wd:Q35657.

?loc <http://schema.org/name> ?place.

} 

"""
print("Results")
run_query(queryString)

Results
[('number', '637')]


1

#### I print some of them to check

In [ ]:
# US State Q35657

queryString = """
SELECT DISTINCT  ?work ?place  WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?work wdt:P17 wd:Q30;
      wdt:P131+ ?loc.

?loc wdt:P31 wd:Q35657.

?loc <http://schema.org/name> ?place.

} LIMIT 300

"""
print("Results")
run_query(queryString)

#### I assume that for "notable" we mean cultural relevance so i reuse the property "heritage designation" (wdt:P1435) found before

In [34]:
# US State Q35657

queryString = """
SELECT DISTINCT ?place (COUNT(?work) AS ?number) WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?work wdt:P17 wd:Q30;
      wdt:P131+ ?loc.

?loc wdt:P31 wd:Q35657.

?loc <http://schema.org/name> ?place.
FILTER (EXISTS{?work wdt:P1435 ?o.})
} 
GROUP BY (?place)
ORDER BY DESC (?number)
LIMIT 1
"""
print("Results")
run_query(queryString)

Results
[('place', 'New York'), ('number', '56')]


1

### Task-4.3 Which U.S. state contains the largest number of building designed by U.S. architects

In [36]:
# US State Q35657

queryString = """
SELECT DISTINCT ?place (COUNT(?work) AS ?number) WHERE { 

?p wdt:P27 wd:Q30;
   wdt:P106 wd:Q42973;
   wdt:P800 ?work.
   
?work wdt:P17 wd:Q30;
      wdt:P131+ ?loc.

?loc wdt:P31 wd:Q35657.

?loc <http://schema.org/name> ?place.
} 
GROUP BY (?place)
ORDER BY DESC (?number)
LIMIT 1
"""
print("Results")
run_query(queryString)

Results
[('place', 'New York'), ('number', '114')]


1

#### We conclude that the state of New York has most buildings designed by US architects among other states